# import

In [2]:
import requests
import pandas as pd
import json
import os
from datetime import datetime, timedelta

import openmeteo_requests

import requests_cache
from retry_requests import retry

import plotly.express as px
import plotly.offline as pyo

import sys
import subprocess
from collections import defaultdict

import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy_playwright.page import PageMethod
from time import sleep



Test de l'adresse avec un exemple de ville

In [3]:
r = requests.get('https://nominatim.openstreetmap.org/search?addressdetails=1&city=Paris&json=json')
r

<Response [200]>

Initialisation de la liste des villes

In [4]:
city = ["Le Mont Saint Michel",
"Saint Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau-du-Haut-Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"La palud sur Verdon",
"Bormes-les-Mimosas",
"Cassis",
"Marseille",
"Aix-en-Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues-Mortes",
"Saintes-Maries-de-la-mer",
"Collioure",
"Carcassonne",
"Foix",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]


In [5]:
# Pour obtenir  une vision plus facile, je mets des définitions aux code météos OMM en limitant les possibilités
weather_code_map = {
    0: "Soleil",
    1: "Soleil",
    2: "Soleil",
    15: "Soleil",
    16: "Nuageux",
    45: "Nuageux",
    48: "Nuageux",
    51: "Bruine",
    53: "Bruine",
    55: "Bruine",
    56: "Bruine",
    57: "Bruine",
    61: "Pluie",
    63: "Pluie",
    65: "Pluie",
    66: "Pluie",
    67: "Pluie",
    71: "Neige",
    73: "Neige",
    75: "Neige",
    77: "Neige",
    80: "Averses",
    81: "Averses",
    82: "Averses",
    85: "Averses",
    86: "Averses",
    95: "Orage",
    96: "Orage",
    99: "Orage"
}

weather_code_map_cplt = {}
last_value = None
for i in range(100):  # de 0 à 99 inclus
    if i in weather_code_map:
        last_value = weather_code_map[i]
    elif last_value is None:
        # avant la première valeur (ex: i < 0), on met la première valeur connue
        last_value = list(weather_code_map.values())[0]
    weather_code_map_cplt[i] = last_value

weather_map_inv = {
    "Soleil": 1,
    "Nuageux": 2,
    "Bruine": 3,
    "Pluie": 4,
    "Neige": 5,
    "Averses": 6,
    "Orage": 7
}
weather_map = {
    1: "Soleil",
    2: "Nuageux",
    3: "Bruine",
    4: "Pluie",
    5: "Neige",
    6: "Averses",
    7: "Orage"
}

In [6]:
# Adresse de l'API pour obtenir les coordonnées des villes
url_city = "https://nominatim.openstreetmap.org/search"

today = datetime.now()
start_date = (today + timedelta(days=4)).strftime('%Y-%m-%d')
end_date = (today + timedelta(days=11)).strftime('%Y-%m-%d')

# Configuration du client Open-Meteo API avec cache et réessai en cas d'erreur
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Adresse de l'API pour la météo
url_meteo = "https://api.open-meteo.com/v1/forecast"

villes = []
meteos = []
id = 0
for ville in city:
    id += 1
    params_nominatim = {
        "format": "json",
        "limit": "1",
        "email": "ristou@free.fr",
        "country": "France",
        "city": ville
    }

    try:
        response_city = requests.get(url_city, params=params_nominatim)
        response_city.raise_for_status()  # Lève une exception pour les erreurs HTTP
        r = response_city.json()

        if r:
            print("ok city", ville)
            villes.append({
                "ville_id": id,
                "nom": ville,
                "latitude": r[0]["lat"],
                "longitude": r[0]["lon"]
            })

            params_meteo = {
                "latitude": r[0]["lat"],
                "longitude": r[0]["lon"],
                "daily": ["weather_code", "apparent_temperature_max", "sunshine_duration", "precipitation_sum", "precipitation_probability_max", "wind_speed_10m_max"],
                "timezone": "auto",
                "start_date": start_date,
                "end_date": end_date,
            }

            responses = openmeteo.weather_api(url_meteo, params=params_meteo)
            if responses:
                response = responses[0]
                daily = response.Daily()

                # Générer les dates
                dates = pd.date_range(
                    start=pd.to_datetime(daily.Time(), unit="s", utc=True),
                    end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=daily.Interval()),
                    inclusive="left")

                # Récupérer les valeurs météorologiques
                weather_code = daily.Variables(0).ValuesAsNumpy()
                temp_max = daily.Variables(1).ValuesAsNumpy()
                sunshine_duration = daily.Variables(2).ValuesAsNumpy()
                precipitation_sum = daily.Variables(3).ValuesAsNumpy()
                precipitation_prob = daily.Variables(4).ValuesAsNumpy()
                wind_speed = daily.Variables(5).ValuesAsNumpy()

                # Créer une ligne pour chaque date
                for i, date in enumerate(dates):
                    meteos.append({
                        "ville_id": id,
                        "date": date.date(),
                        "condition": weather_code[i],
                        "temperature": temp_max[i],
                        "duree_ensoleillement": sunshine_duration[i],
                        "pluie": precipitation_sum[i],
                        "vent": wind_speed[i]
                    })
            else:
                print("erreur météo : ", ville)
        else:
            print("erreur city : ", ville)
    except requests.exceptions.RequestException as e:
        print(f"Erreur de requête pour {ville}: {e}")

df_villes = pd.DataFrame(villes)
df_meteos = pd.DataFrame(meteos)

ok city Le Mont Saint Michel
ok city Saint Malo
ok city Bayeux
ok city Le Havre
ok city Rouen
ok city Paris
ok city Amiens
ok city Lille
ok city Strasbourg
ok city Chateau-du-Haut-Koenigsbourg
ok city Colmar
ok city Eguisheim
ok city Besancon
ok city Dijon
ok city Annecy
ok city Grenoble
ok city Lyon
ok city La palud sur Verdon
ok city Bormes-les-Mimosas
ok city Cassis
ok city Marseille
ok city Aix-en-Provence
ok city Avignon
ok city Uzes
ok city Nimes
ok city Aigues-Mortes
ok city Saintes-Maries-de-la-mer
ok city Collioure
ok city Carcassonne
ok city Foix
ok city Toulouse
ok city Montauban
ok city Biarritz
ok city Bayonne
ok city La Rochelle


In [7]:
for v in villes:
    params_reverse = {
        "lat": v["latitude"],
        "lon": v["longitude"],
        "format": "json",
        "zoom": 10,
        "addressdetails": 1,
        "email": "ristou@free.fr"
    }

    try:
        response_rev = requests.get("https://nominatim.openstreetmap.org/reverse", params=params_reverse)
        response_rev.raise_for_status()

        # Vérifier si la réponse contient bien du JSON
        if "application/json" not in response_rev.headers.get("Content-Type", ""):
            print(f"⚠️ Réponse non JSON pour {v['nom']} : {response_rev.text[:100]}")
            continue

        data_rev = response_rev.json()

        city_name = data_rev.get("address", {}).get("city") or \
                    data_rev.get("address", {}).get("town") or \
                    data_rev.get("address", {}).get("village")

        if city_name:
            print(f"🔍 Vérif : {v['nom']} → {city_name}")
            if city_name.lower() != v["nom"].lower():
                print(f"⚠️ Mismatch possible : {v['nom']} vs {city_name}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur réseau pour {v['nom']}: {e}")
    except ValueError:
        print(f"❌ Erreur JSON vide pour {v['nom']}: {response_rev.text[:100]}")


🔍 Vérif : Le Mont Saint Michel → Le Mont-Saint-Michel
⚠️ Mismatch possible : Le Mont Saint Michel vs Le Mont-Saint-Michel
🔍 Vérif : Saint Malo → Saint-Malo
⚠️ Mismatch possible : Saint Malo vs Saint-Malo
🔍 Vérif : Bayeux → Bayeux
🔍 Vérif : Le Havre → Le Havre
🔍 Vérif : Rouen → Rouen
🔍 Vérif : Paris → Paris
🔍 Vérif : Amiens → Amiens
🔍 Vérif : Lille → Lille
🔍 Vérif : Strasbourg → Strasbourg
🔍 Vérif : Chateau-du-Haut-Koenigsbourg → Orschwiller
⚠️ Mismatch possible : Chateau-du-Haut-Koenigsbourg vs Orschwiller
🔍 Vérif : Colmar → Colmar
🔍 Vérif : Eguisheim → Eguisheim
🔍 Vérif : Besancon → Besançon
⚠️ Mismatch possible : Besancon vs Besançon
🔍 Vérif : Dijon → Dijon
🔍 Vérif : Annecy → Annecy
🔍 Vérif : Grenoble → Grenoble
🔍 Vérif : Lyon → Lyon
🔍 Vérif : La palud sur Verdon → La Palud-sur-Verdon
⚠️ Mismatch possible : La palud sur Verdon vs La Palud-sur-Verdon
🔍 Vérif : Bormes-les-Mimosas → Bormes-les-Mimosas
🔍 Vérif : Cassis → Cassis
🔍 Vérif : Marseille → Marseille
🔍 Vérif : Aix-en-Provence → 

Après vérification Orschwiller est à côté du Château du Haut Koenigsbourg

In [8]:
df_villes

,ville_id,nom,latitude,longitude
0,1,Le Mont Saint Michel,48.6355232,-1.5102571
1,2,Saint Malo,48.6495180,-2.0260409
2,3,Bayeux,49.2764624,-0.7024738
3,4,Le Havre,49.4938975,0.1079732
4,5,Rouen,49.4404591,1.0939658
5,6,Paris,48.8534951,2.3483915
6,7,Amiens,49.8941708,2.2956951
7,8,Lille,50.6365654,3.0635282
8,9,Strasbourg,48.5846140,7.7507127
9,10,Chateau-du-Haut-Koenigsbourg,48.2495226,7.3454923


In [9]:
df_villes['latitude2'] = pd.to_numeric(df_villes['latitude'], errors='coerce')
df_villes['longitude2'] = pd.to_numeric(df_villes['longitude'], errors='coerce')

fig = px.scatter_map(
    df_villes,
    lat="latitude2",
    lon="longitude2",
    hover_name="nom",
    hover_data={
        "ville_id": False,
        "nom": True,
        "latitude2": False,
        "longitude2": False
    },
    zoom=4,
    title="Emplacement des villes"
)

# 5. Affichez la carte
fig.show()


In [10]:
df_meteos["condition"]=df_meteos["condition"].map(weather_code_map_cplt).map(weather_map_inv)
df_meteos

,ville_id,date,condition,temperature,duree_ensoleillement,pluie,vent
0,1,2025-11-15,6,12.147470,3137.130859,0.7,26.178707
1,1,2025-11-16,1,8.873925,11569.625977,0.2,28.673721
2,1,2025-11-17,1,6.984726,28183.871094,0.0,7.952660
3,1,2025-11-18,3,5.724897,22818.716797,0.7,33.563122
4,1,2025-11-19,3,3.097008,28047.734375,0.5,34.979332
...,...,...,...,...,...,...,...
275,35,2025-11-18,3,7.491781,14138.933594,0.9,23.710857
276,35,2025-11-19,1,4.286366,29535.480469,0.0,26.042212
277,35,2025-11-20,3,6.726829,23151.716797,0.3,22.197729
278,35,2025-11-21,1,10.297436,28882.613281,0.0,18.775303


In [11]:
df_villes.to_csv("data//villes.csv", index=False,sep=';')

df_meteos.to_csv("data//meteos.csv", index=False, sep=';')

df_condition = pd.DataFrame(list(weather_map_inv.items()), columns=["description", "code"])
df_condition.to_csv("data//condition.csv", index=False, sep=';')


Lancement du scrapping du site booking

Sélection de la même période. Je limite les chambres entre 100 et 250€ la nuit.

In [13]:
#Lancement 1 par 1 pour réduire les délais
for i in range(len(df_villes)):
    df_villes[['nom','latitude','longitude']][i:i+1].to_json("villes.json", orient="records", force_ascii=False, indent=4)
    # print(df_villes['nom'][i:i+1])
    !python booking_lat_lon.py villes.json


Conversion des fichiers json en dataframe puis csv

In [14]:
# Chemin du répertoire contenant les fichiers JSON
repertoire = 'booking_results'

# Liste pour stocker les DataFrames
dataframes = []

# Parcourir les fichiers dans le répertoire
for fichier in os.listdir(repertoire):
    if fichier.endswith('.json'):
        chemin_fichier = os.path.join(repertoire, fichier)
        try:
            with open(chemin_fichier, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # Convertir les données JSON en DataFrame
                df = pd.DataFrame(data)
                dataframes.append(df)
        except Exception as e:
            print(f"Erreur avec le fichier {fichier}: {e}")


# Concaténer tous les DataFrames en un seul
df_combine = pd.concat(dataframes, ignore_index=True)

# Fusionner les deux DataFrames
df_hotels = pd.merge(
    df_combine,
    df_villes[['nom', 'ville_id']],
    left_on='ville', 
    right_on='nom',
    how='left'
)

df_hotels.drop('nom', axis=1, inplace=True)

# Afficher le DataFrame combiné avec ville_id
df_hotels


,ville,name,description,url,rating,latitude,longitude,prix,ville_id
0,Aigues-Mortes,Hotel De La Pyramide,"Situé sur les rives de l'étang de Thau, l'Hôte...",https://www.booking.com/hotel/fr/de-la-pyramid...,"Avec une note de 9,2",43.41748842190084,3.5957184433937073,€ 724,26
1,Aigues-Mortes,Abrivado,"Situé dans le parc régional de Camargue, l'Abr...",https://www.booking.com/hotel/fr/abrivado.fr.h...,"Avec une note de 8,8",43.45033695444248,4.426224231719971,€ 784,26
2,Aigues-Mortes,La Maison de Lyna - Suites de charme intra-mur...,L’établissement La Maison de Lyna - Suites de ...,https://www.booking.com/hotel/fr/la-maison-de-...,"Avec une note de 9,0",43.5660175,4.1923532,€ 1 019,26
3,Aigues-Mortes,Hôtel Horizon Resort Massane & Spa,L’établissement Hôtel Horizon Resort Massane &...,https://www.booking.com/hotel/fr/horizon-resor...,"Avec une note de 9,1",43.649344153164,4.001000549252,€ 1 011,26
4,Aigues-Mortes,La Villa Mazarin,La Villa Mazarin vous accueille dans un bâtime...,https://www.booking.com/hotel/fr/la-villa-maza...,"Avec une note de 9,2",43.56498661964503,4.191751956841472,€ 929,26
...,...,...,...,...,...,...,...,...,...
688,Uzes,Hotel du Soleil et Spa,L'Hôtel du Soleil & Spa est situé au bout d’un...,https://www.booking.com/hotel/fr/du-soleil-sai...,"Avec une note de 8,5",43.78507429014072,4.832069277763367,€ 853,24
689,Uzes,ibis Avignon Centre Pont De L'Europe,"Situé à 10 minutes à pied du centre d'Avignon,...",https://www.booking.com/hotel/fr/ibis-avignon-...,"Avec une note de 7,7",43.94363815379145,4.79828953742981,€ 712,24
690,Uzes,Kyriad Nîmes Ouest A9,Idéalement situé à 10 minutes en voiture du ce...,https://www.booking.com/hotel/fr/kyriad-na-r-m...,"Avec une note de 7,8",43.81470734512981,4.344832599163055,€ 711,24
691,Uzes,Ibis Budget Bollene,L'Ibis Budget Bollene vous accueille à Bollène...,https://www.booking.com/hotel/fr/ibis-budget-b...,"Avec une note de 7,9",44.289398,4.7524370000000005,€ 710,24


In [15]:
df_hotels.to_csv("data//hotels.csv", index=False, sep=';', encoding='utf-8-sig')


In [16]:
# Calcul de la moyenne par ville
df_meteos_moyenne = df_meteos.groupby('ville_id').mean(numeric_only=True).round(0).reset_index()

# Calcul de la description la plus fréquente par ville
df_meteos_mode = df_meteos.groupby('ville_id')['condition'].apply(lambda x: x.mode()[0]).reset_index()

# Fusionner les deux DataFrames
df_meteos_general = pd.merge(df_meteos_moyenne, df_meteos_mode, on='ville_id')

df_meteos_general['condition_label']=df_meteos_general['condition_x'].map(weather_map)

In [17]:
# Fusionner les DataFrames
df_merged = pd.merge(df_villes, df_meteos_general, on='ville_id')

df_merged['latitude'] = pd.to_numeric(df_merged['latitude'], errors='coerce')
df_merged['longitude'] = pd.to_numeric(df_merged['longitude'], errors='coerce')

df_merged

# Créer la carte pour les températures moyennes
map_temp_moy = px.scatter_map(
    df_merged,
    lat="latitude",
    lon="longitude",
    size="temperature",
    hover_name="nom",
    hover_data={
        "latitude": False,
        "longitude": False,
        "temperature": True,
        "condition_label": True,
        "pluie":True,
        "vent":True
    },
    color="temperature",
    color_continuous_scale="Bluered",
    zoom=4,
    map_style="open-street-map",
    title="Température moyenne séjour"
)

# Sauvegarder et afficher la carte
pyo.plot(map_temp_moy, filename='image//carte_temp_moy.html', auto_open=True)


'image//carte_temp_moy.html'